### Environment Setup

In [ ]:
import pandas as pd
import datetime
import yfinance as yf
import backtrader as bt
import numpy as np
import warnings

warnings.filterwarnings("ignore")

### Parameters Setup

In [ ]:
# Date range
start = '2013-01-01'
end = '2024-09-30'

benchmark_symbol = 'SPY'

# Tickers of assets
stocks_1 = ['JCI', 'TGT', 'CMCSA', 'CPB', 'MO', 'APA', 'MMC', 'JPM',
          'ZION', 'PSA', 'BAX', 'BMY', 'LUV', 'PCAR', 'TXT', 'TMO',
          'DE', 'MSFT', 'HPQ', 'SEE', 'VZ', 'CNP', 'NI', 'T', 'BA']
industry_1 =  ['Consumer Discretionary','Consumer Discretionary',
                              'Consumer Discretionary', 'Consumer Staples',
                              'Consumer Staples','Energy','Financials',
                              'Financials','Financials','Financials',
                              'Health Care','Health Care','Industrials','Industrials',
                              'Industrials','Health Care','Industrials',
                              'Information Technology','Information Technology',
                              'Materials','Telecommunications Services','Utilities',
                              'Utilities','Telecommunications Services','Financials']
stocks_2 = [
            'AMAT', 'AMD','AVGO',  
            'BAC','BKR','BMY','BSX',
            'C','CMCSA',
            'CSCO','CSX','CVS','CVX',
            'DIS','DVN',
            'FCX','FNF','GEN','GILD',
            'GM','HAL','HPQ','INTC',
            'IPG','JNJ','KDP',
            'KKR','KMI','KO',
            'MDLZ','MO','MRK','MRO',
            'MRVL','MSFT','MU','NEM',
]

industry_2 = [
        'Technology','Technology','Technology',
        'Financial Services','Energy','Healthcare','Healthcare',
        'Financial Services','Communication Services',
        'Technology','Industrials','Healthcare','Energy',
        'Communication Services','Energy',
        'Basic Materials','Financial Services','Technology','Healthcare',
        'Consumer Cyclical','Energy','Technology','Technology',
        'Communication Services','Healthcare','Consumer Defensive',
        'Financial Services','Energy','Consumer Defensive',
        'Consumer Defensive','Consumer Defensive','Healthcare','Energy',
        'Technology','Technology','Technology','Basic Materials',
]

ETF_BOND = ['HYG','LQD','TLT',
           ]
industry_BOND = ['Bond','Bond','Bond',
                ]

ETF_COMM = [
    'DBA','GLD','SLV','XLE','XME','XOP',
]

industry_COMM=[
    'Commodity','Commodity','Commodity','Commodity','Commodity','Commodity',
]

ETF_list = ['HYG','LQD','TLT',
           'DBA',
           'GLD','SLV','XLE','XME','XOP','XBI','EFA',
            'EWW','EWZ','VNQ','XHB',
            'IWM','QQQ']
industry_etf =  ['Bond','Bond','Bond',
               'Commodity',
               'Commodity','Commodity','Commodity','Commodity','Commodity',
               'Healthcare',
               'International','International','International',
               'Real Estate','Real Estate',
               'US Major','US Major',
               ]

assets = stocks_2 + ETF_BOND + ETF_COMM + [benchmark_symbol]

asset_classes_dict = {'Assets': stocks_2 + ETF_BOND + ETF_COMM, 
                 'Industry': industry_2+industry_BOND+industry_COMM}

assets.sort()

### Download Data
Full data download from yfinance to **prices** dataframe

In [ ]:
# Downloading data
prices = yf.download(assets, start=start, end=end)
prices = prices.dropna()

In [ ]:
display(prices)

In [ ]:
############################################################
# Calculate assets returns
############################################################

# pd.options.display.float_format = '{:.4%}'.format

data = prices.loc[:, ('Adj Close', slice(None))]
data.columns = assets
data = data.drop(columns=[benchmark_symbol]).dropna()
returns = data.pct_change().dropna()
display(returns.index[0], returns.index[-1])

In [ ]:
#  all test is from the 1004th day and finish at the last testdata day.
#
start_test = 1004
end_test = prices.shape[0] - 1
test_size = 1000
print(f" Testing data from {start_test} to {end_test}")

### Building the Backtest Function with Backtrader

In [ ]:
############################################################
# Defining the backtest function 
############################################################

def backtest(datas, strategy, start, end, plot=False, **kwargs):
    cerebro = bt.Cerebro()

    # print(datas)
    print(f"start={start} - end={end}")
    # Here we add transaction costs and other broker costs
    cerebro.broker.setcash(1000000.0)
    cerebro.broker.setcommission(commission=0.005) # Commission 0.5%
    cerebro.broker.set_slippage_perc(0.005, # Slippage 0.5%
                                     slip_open=True,
                                     slip_limit=True,
                                     slip_match=True,
                                     slip_out=False)
    for data in datas:
        cerebro.adddata(data)

    # Here we add the indicators that we are going to store
    cerebro.addanalyzer(bt.analyzers.TimeReturn, timeframe=bt.TimeFrame.Days)
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, riskfreerate=0.0)
    cerebro.addanalyzer(bt.analyzers.Returns)
    cerebro.addanalyzer(bt.analyzers.DrawDown)
    cerebro.addstrategy(strategy, **kwargs)
    cerebro.addobserver(bt.observers.Value)
    cerebro.addobserver(bt.observers.DrawDown)
    results = cerebro.run(stdstats=False)
    if plot:
        print(f"backtest.plot: {start}-{end}")
        cerebro.plot(iplot=False, start=start, end=end)
    return results[0]


### Building Data Feeds for Backtesting
**asset_prices** = list of all asset except 'SPY' in the bt.feeds of *OHLC + Volume*    
**benchmark**  = 'SPY' *OHLC+Volume* in bt.feeds

In [ ]:
############################################################
# Create objects that contain the prices of assets
############################################################
# Creating Assets bt.feeds
assets_prices = []
for i in assets:
    if i != benchmark_symbol:
        prices_ = prices.drop(columns='Adj Close').loc[:, (slice(None), i)].dropna()
        prices_.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
        print(f"{i}: \n", prices_)
        assets_prices.append(bt.feeds.PandasData(dataname=prices_, plot=False))
        
print(assets_prices)

# Creating Benchmark bt.feeds        
prices_ = prices.drop(columns='Adj Close').loc[:, (slice(None), benchmark_symbol)].dropna()
prices_.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
display(benchmark_symbol)
display(prices_)
benchmark = bt.feeds.PandasData(dataname=prices_, plot=False)

display(prices_.head())

In [ ]:
(slice(None), benchmark_symbol)

In [ ]:
tt = prices.drop(columns='Adj Close').loc[:, (slice(None), benchmark_symbol)].dropna()
display(tt.columns)
display(prices_)

### Buy and Hold for the BenchMark 

In [ ]:
############################################################
# Building the Buy and Hold strategy
############################################################

class BuyAndHold(bt.Strategy):

    def __init__(self):
        self.counter = 0

    def next(self):
        if self.counter >= start_test:
            if self.getposition(self.data).size == 0:
                self.order_target_percent(self.data, target=0.99)
        self.counter += 1 

In [ ]:
print(start_test, end_test,benchmark )

In [ ]:
############################################################
# Run the backtest for the bench mark
############################################################
%matplotlib inline

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (10, 6) # (w, h)
plt.plot() # We need to do this to avoid errors in inline plot

# dd, cagr, sharpe = backtest([benchmark],
benchmark0 = backtest([benchmark],
                            BuyAndHold,
                            start=start_test,
                            end=end_test,
                            plot=True)

In [ ]:
def RetreiveStats(bt_result, rm, obj, r_int):
    dd = bt_result.analyzers.drawdown.get_analysis()['max']['drawdown']
    cagr= bt_result.analyzers.returns.get_analysis()['rnorm100']
    sharpe =bt_result.analyzers.sharperatio.get_analysis()['sharperatio']

    return {'Risk_measure':rm, 'Objective':obj, 'R_Interval': r_int, 'Max DrawDown':dd/100.0, 'CAGR': cagr/100.0, 'Sharpe Ratio':sharpe}

In [ ]:
metric_list = []

metric_list.append(RetreiveStats(benchmark0, benchmark_symbol, 'N/A', 'N/A'))
display(metric_list)

In [ ]:
#
# Retreive Daily Return from bt
#
def RetreiveDailyReturn(bt_result, s_name):
    tret_analyzer = bt_result.analyzers.getbyname('timereturn')
    ret_ = tret_analyzer.get_analysis()
    return pd.DataFrame(ret_.items(), columns=['Date', s_name])


In [ ]:
BM_ret = RetreiveDailyReturn(benchmark0, benchmark_symbol)
print(BM_ret)
BM_ret.to_csv(f"{benchmark_symbol}_ret.csv", index=False)

### Rebalancing Monthly, Quarterly, Semiannually using Riskfolio-Lib

In [ ]:
############################################################
# Selecting Dates for Rebalancing
############################################################

def SelectIndex(price_in, mode="Q"):
    # Selecting last day of month of available data
    index_Month = price_in.groupby([price_in.index.year, price_in.index.month]).tail(1).index
    # print("Monthly: ", index_Month)
    index_daily = price_in.index
    # print("Daily: ", index_daily)

    # Quarterly Dates
    index_Quater = [x for x in index_Month if float(x.month) % 3.0 == 0 ] 
    # print("Quarterly: ", index_Quater)

    # Semi-Annually Dates
    index_Semi = [x for x in index_Month if float(x.month) % 6.0 == 0 ] 
    # print("Semi-Annually: ", index_Semi)

    match mode:
        case "Q": index=index_Quater
        case "M": index = index_Month
        case "D": index = index_daily
        case "S": index = index_Semi
    # Dates where the strategy will be backtested
    index_ = [index_daily.get_loc(x) for x in index if index_daily.get_loc(x) >= start_test]
    return index_

In [ ]:
rebalance_index = {}

In [ ]:
rebalance_index["M"] = SelectIndex(returns, mode="M")
rebalance_index["Q"] = SelectIndex(returns, mode="Q")
rebalance_index["S"] = SelectIndex(returns, mode="S")

In [ ]:
ret = returns.reset_index()
display(ret.iloc[rebalance_index["M"]])

In [ ]:
display(ret.iloc[rebalance_index["Q"]])

In [ ]:
display(ret.iloc[rebalance_index["S"]])

In [ ]:
###########################################################
# Building Constraints
############################################################

asset_classes = pd.DataFrame(asset_classes_dict)
asset_classes = asset_classes.sort_values(by=['Assets'])
# print(asset_classes)
print(asset_classes['Industry'].unique())

constraints = {'Disabled': [False, False, True],
               'Type': ['All Assets', 'All Classes', 'All Classes'],
               'Set': ['', 'Industry', 'Industry'],
               'Position': ['', '', ''],
               'Sign': ['<=', '<=', '>='],
               'Weight': [0.10, 0.20, 0.03],
               'Type Relative': ['', '', ''],
               'Relative Set': ['', '', ''],
               'Relative': ['', '', ''],
               'Factor': ['', '', '']}

constraints = pd.DataFrame(constraints)

display(constraints)

In [ ]:
############################################################
# Building constraint matrixes for Riskfolio Lib
############################################################

import riskfolio as rp

A, B = rp.assets_constraints(constraints, asset_classes)

In [ ]:
############################################################
# Building View for Black Litterman
############################################################
views = {'Disabled': [False, False, False],
         'Type': ['Classes', 'Classes', 'Classes'],
         'Set': ['Industry', 'Industry', 'Industry'],
         'Position': ['Technology', 'Energy', 'Healthcare'],
         'Sign': ['>=', '>=', '>='],
         'Weight': [0.20, 0.1, 0.09], # Annual terms 
         'Type Relative': ['Classes', 'Classes', 'Classes'],
         'Relative Set': ['Industry', 'Industry', 'Industry'],
         'Relative': ['Financial Services', 'Commodity', 'Consumer Defensive']}

views = pd.DataFrame(views)

display(views)

In [ ]:
asset_classes

In [ ]:
P, Q = rp.assets_views(views, asset_classes)

display(pd.DataFrame(P.T))
display(pd.DataFrame(Q))

* The optimization is based on the returns of previous 1000 days from last date of each quarter, which is about 4 years

In [ ]:
plotFlag = True

# Risk Measures available:
#
# 'MV': Standard Deviation.
# 'MAD': Mean Absolute Deviation.
# 'MSV': Semi Standard Deviation.
# 'FLPM': First Lower Partial Moment (Omega Ratio).
# 'SLPM': Second Lower Partial Moment (Sortino Ratio).
# 'CVaR': Conditional Value at Risk.
# 'EVaR': Entropic Value at Risk.
# 'WR': Worst Realization (Minimax)
# 'MDD': Maximum Drawdown of uncompounded cumulative returns (Calmar Ratio).
# 'ADD': Average Drawdown of uncompounded cumulative returns.
# 'CDaR': Conditional Drawdown at Risk of uncompounded cumulative returns.
# 'EDaR': Entropic Drawdown at Risk of uncompounded cumulative returns.
# 'UCI': Ulcer Index of uncompounded cumulative returns.

rms = ['MV', 'MAD', 'MSV', 'FLPM', 'SLPM', 'CVaR',
       'EVaR', 'WR', 'MDD', 'ADD', 'CDaR', 'UCI', 'EDaR']
# rms = ['MV']

# Objective Functions 
objectives = ['Sharpe', 'MinRisk', 'MaxRet']
# objectives = ['Sharpe', 'MaxRet']

# rebalance interval: Monthly, Quarterly, Semiannually
reb_interval = ["M","Q","S"]
# reb_interval = ["Q","S"]

In [ ]:
def my_optimize(ob, rr, rrmm, YY, AA, BB, PP, QQ):
    # Building the portfolio object
    port = rp.Portfolio(returns=YY)
    
    # port.solvers = ['MOSEK']
    port.alpha = 0.05
    model='BL' # Could be Classic (historical), BL (Black Litterman) or FM (Factor Model)
    hist = False # Use historical scenarios for risk measures that depend on scenarios
    rf = 0 # Risk free rate
    l = 0 # Risk aversion factor, only useful when obj is 'Utility'
    
    # Add portfolio constraints
    port.ainequality = AA
    port.binequality = BB
    
    # Calculating optimum portfolio
    
    # Select method and estimate input parameters:
    
    method_mu='hist' # Method to estimate expected returns based on historical data.
    method_cov='hist' # Method to estimate covariance matrix based on historical data.
    
    port.assets_stats(method_mu=method_mu, method_cov=method_cov)
    
    # Estimate optimal portfolio:
    w = port.optimization(model='Classic', rm=rm, obj=obj, rf=rf, l=l, hist=True)
    
    # Estimate Black Litterman inputs:
    port.blacklitterman_stats(PP, QQ/252, rf=rf, w=w, delta=None, eq=True)
    
    if rm == 'MV':
        hist = False
    else:
        hist = True
    w_bl = port.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)

    return w_bl

In [ ]:
%%time
############################################################
# Building a loop that estimate optimal portfolios on
# rebalancing dates
############################################################

models = {}

for obj in objectives:
    models[obj] = {}
    for r in reb_interval:
        models[obj][r] = {}
        for rm in rms:
            print(obj, ",", r, ",", rm)
            weights = pd.DataFrame([])
            for i in rebalance_index[r]:
                Y = returns.iloc[i-test_size:i,:] # taking last 4 years (250 trading days per year)
                # display("from ", Y.index[0], " to ", Y.index[-1])
                # display(Y.head())

                # Building the portfolio object
                port = rp.Portfolio(returns=Y)

                # port.solvers = ['MOSEK']
                port.alpha = 0.05
                model='BL' # Could be Classic (historical), BL (Black Litterman) or FM (Factor Model)
                hist = False # Use historical scenarios for risk measures that depend on scenarios
                rf = 0 # Risk free rate
                l = 0 # Risk aversion factor, only useful when obj is 'Utility'

                # Add portfolio constraints
                port.ainequality = A
                port.binequality = B
                
                # Calculating optimum portfolio
        
                # Select method and estimate input parameters:
        
                method_mu='hist' # Method to estimate expected returns based on historical data.
                method_cov='hist' # Method to estimate covariance matrix based on historical data.
        
                port.assets_stats(method_mu=method_mu, method_cov=method_cov)
                
                # Estimate optimal portfolio:
                w = port.optimization(model='Classic', rm=rm, obj=obj, rf=rf, l=l, hist=True)
                
                # Estimate Black Litterman inputs:
                port.blacklitterman_stats(P, Q/252, rf=rf, w=w, delta=None, eq=True)

                if rm == 'MV':
                    hist = False
                else:
                    hist = True
                w = port.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)

                if w is None:
                    w = weights.tail(1).T
                # display(w)
                weights = pd.concat([weights, w.T], axis = 0)
            
            models[obj][r][rm] = weights.copy()
            models[obj][r][rm].index = rebalance_index[r]

In [ ]:
pd.options.display.float_format = '{:.4%}'.format

In [ ]:
for obj in objectives:
    for r in reb_interval:
        for rm in rms:
            wght = models[obj][r][rm]
            display(f'obj={obj}, int={r}, rm={rm}')
            display(wght.index)
            display(wght.head(5))

In [ ]:
display(models['Sharpe']['Q']['MV'])

In [ ]:
############################################################
# Building the Asset Allocation Class
############################################################

class AssetAllocation(bt.Strategy):

    def __init__(self):
        print("AssetAllocation: ", assets)
        j = 0
        for i in assets:
            setattr(self, i, self.datas[j])
            # print(f"{j},{i}\n", self.datas[j])
            j += 1
        
        self.counter = 0
        
    def next(self):
        if self.counter in weights.index.tolist():
            for i in assets:
#                 print(f"counter: {self.counter},{i}")
#                 print(" weights size: ", weights.shape)
                w = weights.loc[self.counter, i]
                self.order_target_percent(getattr(self, i), target=w)
        self.counter += 1

In [ ]:
%%time
############################################################
# Backtesting All Strategy
############################################################
for obj in objectives:
    for r in reb_interval:
        for rm in rms:
            
            assets = returns.columns.tolist()
            weights = models[obj][r][rm]
        
            result0 = backtest(assets_prices,
                            AssetAllocation,
                            start=start_test,
                            end=end_test,
                            plot=plotFlag)
           
            metric_list.append(RetreiveStats(result0, rm, obj, r))
            display(metric_list)
        
            Dret = RetreiveDailyReturn(result0, 'Return')
            print(Dret)
            Dret.to_csv(f"{obj}_{r}_{rm}_ret.csv", index=False)
            
            ############################################################
            # Plotting the composition of the last MV portfolio
            ############################################################
        
            w = pd.DataFrame(models[obj][r][rm].iloc[-1,:])

            if plotFlag:
                # We need matplotlib >= 3.3.0 to use this function
                ax = rp.plot_pie(w=w, title=f'{obj}-{rm}-{r}', others=0.05, nrow=25, cmap = "tab20",
                                height=6, width=10, ax=None)
        
                # w.plot.pie(subplots=True, figsize=(8, 8))
            
            ############################################################
            # Composition per Industry
            ############################################################
        
            w_classes = pd.concat([asset_classes.set_index('Assets'), w], axis=1)
            w_classes = w_classes.groupby(['Industry']).sum()
            w_classes.columns = ['weights']
            display(w_classes)

In [ ]:
metric_df = pd.DataFrame(metric_list)
display(metric_df[metric_df['Risk_measure']==benchmark_symbol])
display(metric_df.sort_values(by=['Sharpe Ratio','CAGR'], ascending=False))

In [ ]:
display(metric_df[metric_df['R_Interval']=='Q'].sort_values(by=['Sharpe Ratio','CAGR'], ascending=False))

In [ ]:
display(metric_df[metric_df['R_Interval']=='S'].sort_values(by=['Sharpe Ratio','CAGR'], ascending=False))

In [ ]:
metric_df.to_csv("Port_Metric.csv", index=False)

In [ ]:
for obj in objectives:
    for r in reb_interval:
        for rm in rms:
            models[obj][r][rm].to_csv(f"weights_{obj}_{r}_{rm}.csv")